In [1]:
import wandb
import os 
import numpy as np
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, storage
from tqdm import tqdm 

/tmp/ipykernel_53009/1617223376.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Initialize Firebase Admin SDK
cred = credentials.Certificate('disaster-master.json')  # Replace with your Firebase Admin SDK key
firebase_admin.initialize_app(cred, {
    'storageBucket': 'disaster-master-59d6f.appspot.com'  # Replace with your storage bucket URL
})

db = firestore.client()
bucket = storage.bucket()

In [3]:
"""
provide a function for this 
Check firestore db 

Check the users table -> check gameEnded true -> log with prefix `All` -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `victim` -> check users.experience.victim true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `volunteer` -> check users.experience.volunteer true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
Check the users table -> Check gameEnded true -> log with prefix `decision_maker` -> check users.experience.decision_maker true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb
"""



'\nprovide a function for this \nCheck firestore db \n\nCheck the users table -> check gameEnded true -> log with prefix `All` -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `victim` -> check users.experience.victim true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `volunteer` -> check users.experience.volunteer true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested`, `WronglyAnswered` , `score` in a list. Take the average and log to wandb\nCheck the users table -> Check gameEnded true -> log with prefix `decision_maker` -> check users.experience.decision_maker true -> store all users. `CorrectlyAnswered`, `GatherAdditionalDataRequested

In [4]:
# Initialize WandB
wandb.init(project="human_study")

def log_to_wandb(prefix, data):
    avg_data = {
        "CorrectlyAnswered": sum(d['CorrectlyAnswered'] for d in data) / len(data),
        "GatherAdditionalDataRequested": sum(d['GatherAdditionalDataRequested'] for d in data) / len(data),
        "WronglyAnswered": sum(d['WronglyAnswered'] for d in data) / len(data),
        "score": sum(d['score'] for d in data) / len(data),
        "total_trees_done":sum(d['trees'] for d in data),
        "avg_trees_done":sum(d['trees'] for d in data) / len(data)
    }
    wandb.log({f"{prefix}/CorrectlyAnswered_avg": avg_data["CorrectlyAnswered"]})
    wandb.log({f"{prefix}/GatherAdditionalDataRequested_avg": avg_data["GatherAdditionalDataRequested"]})
    wandb.log({f"{prefix}/WronglyAnswered_avg": avg_data["WronglyAnswered"]})
    wandb.log({f"{prefix}/score_avg": avg_data["score"]})
    wandb.log({f"{prefix}/total_trees_done": avg_data["total_trees_done"]})
    wandb.log({f"{prefix}/avg_trees_done": avg_data["avg_trees_done"]})
    
    # Log individual data points
    step = 0
    for i, d in enumerate(data):
        wandb.log({f"{prefix}/CorrectlyAnswered": d["CorrectlyAnswered"],"step":step})
        wandb.log({f"{prefix}/GatherAdditionalDataRequested": d["GatherAdditionalDataRequested"],"step":step})
        wandb.log({f"{prefix}/WronglyAnswered": d["WronglyAnswered"],"step":step})
        wandb.log({f"{prefix}/score": d["score"],"step":step})
        wandb.log({f"{prefix}/trees_done": d["trees"],"step":step})
        step += 1

def process_users_data():
    users_ref = db.collection('users')
    users = users_ref.where('gameEnded', '==', True).stream()

    all_data = []
    victim_data = []
    volunteer_data = []
    decision_maker_data = []

    for user in users:
        user_data = user.to_dict()
        user_stats = {
            "CorrectlyAnswered": user_data.get("CorrectlyAnswered", 0),
            "GatherAdditionalDataRequested": user_data.get("GatherAdditionalDataRequested", 0),
            "WronglyAnswered": user_data.get("WronglyAnswered", 0),
            "score": user_data.get("score", 0),
            "trees":user_data.get("current_tree_level",0)
        }
        all_data.append(user_stats)

        if user_data.get("experience", {}).get("victim"):
            victim_data.append(user_stats)
        
        if user_data.get("experience", {}).get("volunteer"):
            volunteer_data.append(user_stats)
        
        if user_data.get("experience", {}).get("decision_maker"):
            decision_maker_data.append(user_stats)

    if all_data:
        log_to_wandb("All", all_data)
    
    if victim_data:
        log_to_wandb("victim", victim_data)
    
    if volunteer_data:
        log_to_wandb("volunteer", volunteer_data)
    
    if decision_maker_data:
        log_to_wandb("decision_maker", decision_maker_data)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: juliangeralddcruz. Use `wandb login --relogin` to force relogin


In [5]:
process_users_data()

/home/julian/miniconda3/envs/pytorch_env/lib/python3.10/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


In [6]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


All/CorrectlyAnswered,▆▇▄▃▆▅▆▇▇▅▆▇▇▅▄▅▅▆▆▅▆▄▇▂▂▄▅▁▄▅▆▇▃▄▇▆█▅▅▄
All/CorrectlyAnswered_avg,▁
All/GatherAdditionalDataRequested,▇▁▁▄▆▂▁▁▁▁█▆▃▅▂▆▁▁▁▆▁▄▁▃▄▅▂▁▂█▂▃▆▆▃▁▁▆▁▃
All/GatherAdditionalDataRequested_avg,▁
All/WronglyAnswered,▃▂▅▄▃▄▃▂▂▄▃▂▂▄▃▄▄▃▃▄▄▅▂▅▅▃▄█▅▄▃▂▄▅▂▃▁▃▃▄
All/WronglyAnswered_avg,▁
All/avg_trees_done,▁
All/score,▆▇▃▅▆▅▆▇▇▅▆▆▇▅▅▅▅▆▆▅▅▄▇▃▃▅▅▁▄▅▆▇▅▄▇▆█▆▆▅
All/score_avg,▁
All/total_trees_done,▁
All/trees_done,▂▁▁▂▂▃▂▁▂▂▁▂▂▁▁▁█▂▃▂▃▁▁▁▁▂▂▁▂▁▁▂▁▁▂▂▁▂▂▅
